In [1]:
import pandas as pd
import io
import requests

url = "https://docs.google.com/spreadsheets/d/e/2PACX-1vTSLyp3Ev2bxe8pnHStFtbdwg-EhvbMEhEUzRsaFjC0sKVQ-OzyD8LbAjM9iYoG2LSmSTAGGFfed4I9/pub?gid=0&single=true&output=csv"

s=requests.get(url).content
df=pd.read_csv(io.StringIO(s.decode('utf-8')))
df.head()

,Level,Title,Text,Source,Type
0,A1,Meine Familie,Zu meiner Familie gehören vier Personen. Die M...,https://german.net/reading/familie/,Reading
1,A1,Maria und ihre Familie,Mein Name ist Maria. Ich bin 30 Jahre alt. Zu ...,https://german.net/reading/marias-familie/,Reading
2,A1,Ich bin Tom,Hallo! Ich bin Tom Maier. Ich bin 13 Jahre alt...,https://german.net/reading/tom/,Reading
3,A1,Freundinnen,Ricarda ist 21 Jahre alt und wohnt in Lübeck. ...,https://german.net/reading/freundinnen/,Reading
4,A1,Einkaufen im Supermarkt,Frau Meier geht heute in den Supermarkt. Ihr M...,https://german.net/reading/einkaufen/,Reading


In [2]:
import urllib
url = "http://www.cis.uni-muenchen.de/~schmid/tools/RFTagger/data/RFTagger.tar.gz"
filename = "RFTagger.tar.gz"
urllib.request.urlretrieve(url, filename)
import tarfile
tar = tarfile.open(filename)
tar.extractall()
tar.close()

In [3]:
from subprocess import run, check_output

def preprocessing(text):
    file = open("RFTagger/temp.txt", "w")
    file.write(text)
    file.close()

    test_tagged = check_output(["cmd/rftagger-german", "temp.txt"], cwd="RFTagger").decode("utf-8").split("\n")

    sent_indecies = [index for index in range(len(test_tagged)) if test_tagged[index] == '']
    sent_indecies = [-1] + sent_indecies

    test_tagged_saparated= []
    for entry in test_tagged:
        temp = entry.split("\t")
        if len(temp) == 3:
            test_tagged_saparated.append([temp[0]] + temp[1].split(".") + [temp[2]])
        else:
            test_tagged_saparated.append([])

    output = []
    for start, end in zip(sent_indecies, sent_indecies[1:]):
        output.append(test_tagged_saparated[start+1: end])

    return output

In [4]:
df["preprocessedText"] = [preprocessing(text) for text in df["Text"]]

In [5]:
display(df.head())
filename = "preprocessed_text_df.csv"
df.to_csv(filename)

,Level,Title,Text,Source,Type,preprocessedText
0,A1,Meine Familie,Zu meiner Familie gehören vier Personen. Die M...,https://german.net/reading/familie/,Reading,"[[[Zu, APPR, zu], [meiner, PRO, Poss, Attr, Da..."
1,A1,Maria und ihre Familie,Mein Name ist Maria. Ich bin 30 Jahre alt. Zu ...,https://german.net/reading/marias-familie/,Reading,"[[[Mein, PRO, Poss, Attr, Nom, Sg, Masc, meine..."
2,A1,Ich bin Tom,Hallo! Ich bin Tom Maier. Ich bin 13 Jahre alt...,https://german.net/reading/tom/,Reading,"[[[Hallo, ITJ, hallo], [!, SYM, Pun, Sent, !]]..."
3,A1,Freundinnen,Ricarda ist 21 Jahre alt und wohnt in Lübeck. ...,https://german.net/reading/freundinnen/,Reading,"[[[Ricarda, N, Name, Nom, Sg, Fem, Ricarda], [..."
4,A1,Einkaufen im Supermarkt,Frau Meier geht heute in den Supermarkt. Ihr M...,https://german.net/reading/einkaufen/,Reading,"[[[Frau, N, Reg, Nom, Sg, Fem, Frau], [Meier, ..."
